In [1]:
# fake_watermark
# https://drive.google.com/file/d/1tDe15P-X38pWtREM5KKCN5eaQ5jCo9WH/view?usp=share_link
!gdown --id 1tDe15P-X38pWtREM5KKCN5eaQ5jCo9WH
!unzip fake_watermark.zip

/usr/local/lib/python3.9/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1tDe15P-X38pWtREM5KKCN5eaQ5jCo9WH
To: /content/fake_watermark.zip
100% 25.5M/25.5M [00:00<00:00, 82.8MB/s]
Archive:  fake_watermark.zip
   creating: fake_watermark/
   creating: fake_watermark/images/
  inflating: fake_watermark/images/0.jpg  
  inflating: fake_watermark/images/1.jpg  
  inflating: fake_watermark/images/2.jpg  
  inflating: fake_watermark/images/3.jpg  
  inflating: fake_watermark/images/4.jpg  
  inflating: fake_watermark/images/5.jpg  
  inflating: fake_watermark/images/6.jpg  
  inflating: fake_watermark/images/7.jpg  
  inflating: fake_watermark/images/8.jpg  
  inflating: fake_watermark/images/9.jpg  
  inflating: fake_watermark/images/10.jpg  
  inflating: fake_watermark/images/11.jpg  
  inflating: f

In [2]:
import os
import cv2

image_folder = "fake_watermark/images"
annotation_folder = "fake_watermark/annotations"

# Get the list of image files
image_files = os.listdir(image_folder)

# Load the annotations for each image
for image_file in image_files:
    # Load the image
    image_path = os.path.join(image_folder, image_file)
    image = cv2.imread(image_path)
    
    # Load the annotation file
    annotation_file = os.path.splitext(image_file)[0] + ".txt"
    annotation_path = os.path.join(annotation_folder, annotation_file)
    with open(annotation_path, "r") as f:
        annotation_lines = f.readlines()

In [3]:
# Parse the annotation lines to extract the bounding boxes and class labels
boxes = []
labels = []
for annotation_line in annotation_lines:
    annotation = annotation_line.strip().split(" ")
    class_label = int(annotation[0])
    x_center = float(annotation[1])
    y_center = float(annotation[2])
    width = float(annotation[3])
    height = float(annotation[4])

    print(x_center, y_center, width, height)
    print(image.shape)
    
    # Convert the bounding box coordinates to top-left and bottom-right coordinates
    x1 = int((x_center - width/2) * image.shape[1])
    y1 = int((y_center - height/2) * image.shape[0])
    x2 = int((x_center + width/2) * image.shape[1])
    y2 = int((y_center + height/2) * image.shape[0])

    print(x1,y1,x2,y2)
    
    # Add the bounding box and class label to the list
    boxes.append([x1, y1, x2, y2])
    labels.append(class_label)

0.26953125 0.033203125 0.541015625 0.541015625
(512, 512, 3)
0 -121 276 155


In [4]:
# Preprocess the image and annotation for training
new_size = (640, 640)
image = cv2.resize(image, new_size)
image = image.astype("float32") / 255.0

# Convert the boxes to the format required by the object detection model (xmin, ymin, xmax, ymax)
boxes = [[box[0], box[1], box[2], box[3]] for box in boxes]

In [5]:
from sklearn.model_selection import train_test_split

# Split the data into training and validation sets
train_images, val_images, train_boxes, val_boxes, train_labels, val_labels = train_test_split(
    images, boxes, labels, test_size=0.2, random_state=42)

NameError: ignored

In [ ]:
# Define the YOLOv5 model
import torch
from models.yolov5 import YOLOv5

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = YOLOv5(num_classes=1).to(device)

# Define the optimizer and loss function
import torch.optim as optim
import torch.nn as nn

optimizer = optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.BCEWithLogitsLoss()

# Define the training loop
def train_one_epoch(model, optimizer, criterion, train_loader, device):
    model.train()
    epoch_loss = 0.0
    for i, (images, boxes, labels) in enumerate(train_loader):
        images = images.to(device)
        boxes = [b.to(device) for b in boxes]
        labels = [l.to(device) for l in labels]

        optimizer.zero_grad()
        outputs = model(images)
        loss = 0.0
        for j, out in enumerate(outputs):
            box_out, cls_out = out
            box_gt = boxes[j]
            cls_gt = labels[j]
            loss += criterion(cls_out, cls_gt) + criterion(box_out, box_gt)

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(train_loader)

# Define the validation loop
def validate(model, criterion, val_loader, device):
    model.eval()
    epoch_loss = 0.0
    with torch.no_grad():
        for i, (images, boxes, labels) in enumerate(val_loader):
            images = images.to(device)
            boxes = [b.to(device) for b in boxes]
            labels = [l.to(device) for l in labels]

            outputs = model(images)
            loss = 0.0
            for j, out in enumerate(outputs):
                box_out, cls_out = out
                box_gt = boxes[j]
                cls_gt = labels[j]
                loss += criterion(cls_out, cls_gt) + criterion(box_out, box_gt)

            epoch_loss += loss.item()

    return epoch_loss / len(val_loader)

# Train the model
from torch.utils.data import DataLoader
from datasets.watermark_dataset import WatermarkDataset

train_dataset = WatermarkDataset(train_images, train_boxes, train_labels)
val_dataset = WatermarkDataset(val_images, val_boxes, val_labels)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)

num_epochs = 10
for epoch in range(num_epochs):
    train_loss = train_one_epoch(model, optimizer, criterion, train_loader, device)
    val_loss = validate(model, criterion, val_loader, device)
    print(f"Epoch {epoch+1} - Train loss: {train_loss:.4f} - Val loss: {val_loss:.4f}")

In [ ]:
# Use the trained model to detect watermarks in test images
import matplotlib.pyplot as plt

def draw_boxes(image, boxes, labels):
    for box, label in zip(boxes, labels):
        x1, y1, x2, y2 = box
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(image, f"Watermark {label}", (x1, y1 - 5), cv

In [ ]:
# Use the trained model to detect watermarks in test images
import cv2

model.eval()

with torch.no_grad():
    for image_path in test_images:
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        height, width = image.shape[:2]

        outputs = model.detect(image)

        boxes = []
        labels = []
        for out in outputs:
            box_out, cls_out = out
            box_out[:, [0, 2]] *= width
            box_out[:, [1, 3]] *= height
            box_out = box_out.int().cpu().numpy().tolist()
            cls_out = cls_out.sigmoid().cpu().numpy().tolist()
            for box, cls in zip(box_out, cls_out):
                if cls > 0.5:
                    boxes.append(box)
                    labels.append(cls)

        draw_boxes(image, boxes, labels)

        plt.imshow(image)
        plt.axis("off")
        plt.show()